# Given 

- `api_response` (such as JSON from a web-service)
- `dimensions`
- `kpis`

# Expected

A list of flattened JSON with the names of each dimension listed out and the kpis extracted from the inner-most values. 

```python
def flatten_response(api_response, dimensions, kpis):
    pass
```

In [58]:
dimensions = ["network", "campaign", "agenda", "date"]
kpis = ["clicks", "impressions", "results"]

api_response = {
  "network": [
      {
          "name": "Foo",
          "campaign": [
              {
                  "name": "C1",
                  "agenda": [
                      {
                          "name": "A1",
                          "date": [
                              {
                                "date": "2012-01-02",
                                "values": [1, 2, 3]
                              }
                          ]
                      }
                  ]
              }
          ]
      },
      {
        "name": "Bar",
        "campaign": [
            {
              "name": "C2",
              "agenda": [
                  {
                      "name": "A2",
                      "date": [
                          {
                            "date": "2012-01-01",
                            "values": [4, 5, 6]
                          },
                          {
                            "date": "2012-03-01",
                            "values": [4, 5, 6]
                          }
                      ]
                  },
                  {
                      "name": "A2-2",
                      "date": [
                          {
                            "date": "2012-01-01",
                            "values": [4, 5, 6]
                          },
                          {
                            "date": "2012-03-01",
                            "values": [4, 5, 6]
                          }
                      ]
                  },
              ]
            },
            {
              "name": "C2-2",
                  "agenda": [
                      {
                          "name": "A3",
                          "date": [
                              {
                                "date": "2014-01-01",
                                "values": [-2, 5, 60]
                              },
                              {
                                "date": "2015-03-01",
                                "values": [4, 10, 6]
                              }
                          ]
                      }
                  ]
            }
         ]
      }
  ]
}

In [64]:

def _flatten_last_dimension(response, dimension, kpis):  
    result = []
    for x in response.get(dimension, []): # assert dimension == last dimension name
        _x = x.copy()
        for i, k in enumerate(kpis): 
            _x[k] = _x['values'][i]
        del _x['values']
        
        result.append(_x)
    
    return result
    

def _flatten_response(dimension_response, dimensions, d_i, kpis):
    result = []

    d = dimensions[d_i]
    for x in dimension_response.get(d, []):
        name = x["name"]
        if d_i == len(dimensions) - 2:  # next iteration will be the last dimension
            resp = _flatten_last_dimension(x, dimensions[d_i+1], kpis)
        else:
            resp = _flatten_response(x, dimensions, d_i+1, kpis)
        for r in resp:
            r[d] = name
            result.append(r)

    return result

def flatten_response(api_response, dimensions, kpis):
    return _flatten_response(api_response, dimensions, 0, kpis)


In [65]:

for r in flatten_response(api_response, dimensions, kpis):
    print(r)

{'date': '2012-01-02', 'clicks': 1, 'impressions': 2, 'results': 3, 'agenda': 'A1', 'campaign': 'C1', 'network': 'Foo'}
{'date': '2012-01-01', 'clicks': 4, 'impressions': 5, 'results': 6, 'agenda': 'A2', 'campaign': 'C2', 'network': 'Bar'}
{'date': '2012-03-01', 'clicks': 4, 'impressions': 5, 'results': 6, 'agenda': 'A2', 'campaign': 'C2', 'network': 'Bar'}
{'date': '2012-01-01', 'clicks': 4, 'impressions': 5, 'results': 6, 'agenda': 'A2-2', 'campaign': 'C2', 'network': 'Bar'}
{'date': '2012-03-01', 'clicks': 4, 'impressions': 5, 'results': 6, 'agenda': 'A2-2', 'campaign': 'C2', 'network': 'Bar'}
{'date': '2014-01-01', 'clicks': -2, 'impressions': 5, 'results': 60, 'agenda': 'A3', 'campaign': 'C2-2', 'network': 'Bar'}
{'date': '2015-03-01', 'clicks': 4, 'impressions': 10, 'results': 6, 'agenda': 'A3', 'campaign': 'C2-2', 'network': 'Bar'}


In [28]:
print(_flatten_last_dimension(api_response['network'][1]['campaign'][1], dimensions[-1], kpis))

[{'date': '2014-01-01', 'clicks': -2, 'impressions': 5, 'results': 60}, {'date': '2015-03-01', 'clicks': 4, 'impressions': 10, 'results': 6}]
